In [40]:
import torch
!pip install gpytorch
import gpytorch
from gpytorch.kernels.rbf_kernel import postprocess_rbf

In [ ]:
class AleboKernel(gpytorch.kernels.Kernel):
  def __init__(self,B,batch_shape):
    super().__init__(has_length=False,ard_num_dims=None,eps=0.0,batch_shape=batch_shape)
    self.d = B.shape[0]
    self.D = B.shape[1]
    self.B = B
    self.Binv = torch.pinverse(B)
    self.dtype = B.dtype
    self.batch_shape = batch_shape

    A = torch.qr(torch.randn(self.D,self.D,dtype = self.dtype))[0]
    ABinv = A[:self.d,:] @ self.Binv
    T = ABinv.t() @ ABinv
    U = torch.cholesky(T,upper = True)
    self.idx = U.nonzero().t().tolist()
    self.Uvec = U[idx].repeat(*batch_shape,1)
    self.register_parameter(name = "Uvec",torch.nn.Parameter(Uvec))
    
  def forward(self,x1,x2,**params):
    U_t = torch.zeros((self.Uvec.shape[:-1],self.d,self.d),dtype = self.dtype)
    U_t[...,self.idx[1],self.idx[0]] = self.Uvec

    z1 = x1 @ U_t
    z2 = x2 @ U_t

    return self.covar_dist(z1,z2,square_dist=True,dist_postprocess_func=postprocess_rbf,**params)

# class AleboGP()


In [37]:
# B = torch.randn((2,3))
# A = torch.qr(torch.randn(3,3))[0]
# ABinv = A[:2,:] @ torch.pinverse(B)
# T = ABinv.t() @ ABinv
# U = torch.cholesky(T,upper = True)
# triu_idx = torch.triu_indices(2,2)
# print(U.shape)
# print(triu_idx.tolist())
# Uvec = U[triu_idx.tolist()].repeat(5,1)
# print(Uvec,U)

torch.Size([2, 2])
[[0, 0, 1], [0, 1, 1]]
tensor([[ 0.7766, -0.0938,  0.5994],
        [ 0.7766, -0.0938,  0.5994],
        [ 0.7766, -0.0938,  0.5994],
        [ 0.7766, -0.0938,  0.5994],
        [ 0.7766, -0.0938,  0.5994]]) tensor([[ 0.7766, -0.0938],
        [ 0.0000,  0.5994]])
